In [1]:
from __future__ import print_function, division
from gurobipy import Model, GRB, quicksum, LinExpr
import networkx as nx
import pickle
import numpy, random

In [2]:
def MIP_IM():
    budget = 25
    m = 100
    epsilon = 1
    
    attributes = ['gender','age', 'region', 'ethnicity']
  
    for attribute in attributes:
        for index in range(20):
            with open('../MIP/data/networks_prob/graph_spa_500_'+str(index)+'.pickle', "rb") as f:
                main_graph = pickle.load(f)

                samples = []
                for j in range(m):
                    G = nx.DiGraph()
                    for u in main_graph.nodes():
                        G.add_node(u)
                    for u,v in main_graph.edges():
                        if main_graph[u][v]['p']> random.random():
                            G.add_edge(u, v)
                    samples.append(G)


                model = Model('fairseed_'+str(index))

                mvars = []
                #active nodes
                avars = []
                #seed nodes
                svars = []
                var_seed_dict = {}
                var_active_dict = {}
                var_mean_dict = {}

                for j in range(len(main_graph.nodes())):
                    s = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                    svars.append(s)
                    var_seed_dict[j] = s

                for sample_index, sample in enumerate(samples):
                    for j in range(len(main_graph.nodes())):
                        a = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                        avars.append(a)
                        var_active_dict[(sample_index,j)] = a    

                mvars.append(avars)
                mvars.append(svars)

                obj_expr = quicksum(avars)
                model.setObjective(obj_expr, GRB.MAXIMIZE)
                model.addConstr(quicksum(svars), GRB.LESS_EQUAL, budget)

                for sample_index, sample in enumerate(samples):
                    for i in range(len(main_graph.nodes())):
                        neighbors = nx.ancestors(sample, i) 
                        e = len(neighbors)
                        ai = var_active_dict[(sample_index,i)]
                        si = var_seed_dict[i]
                        neighbors_active_vars = []
                        neighbors_seed_vars = []
                        neighbors_active_vars.append(((e+1), ai))
                        neighbors_seed_vars.append(si)
                        for neighbor in neighbors:
                            neighbors_active_vars.append(((e+1), var_active_dict[(sample_index,neighbor)]))
                            neighbors_seed_vars.append(var_seed_dict[neighbor])
                        seed_neighbors = quicksum(neighbors_seed_vars)
                        model.addConstr(ai, GRB.LESS_EQUAL, seed_neighbors)
                        model.addConstr(seed_neighbors, GRB.LESS_EQUAL, LinExpr(neighbors_active_vars))


                labels = nx.get_node_attributes(main_graph, attribute)
                label_dict = {}
                for i in range(len(main_graph.nodes())):
                    label = labels[i].encode('utf-8')
                    if label in label_dict.keys():
                        label_dict[label].append(i)
                    else:
                        label_dict[label] = [i]     

                for label in label_dict.keys():
                    seed_vars = []
                    label_size = len(label_dict[label])
                    frac = float(label_size)/float(len(main_graph.nodes()))
                    for node in label_dict[label]:
                        seed_vars.append(var_seed_dict[node])
                    label_budget = budget*frac
                    expr = quicksum(seed_vars)
                    model.addConstr(expr, GRB.LESS_EQUAL, label_budget+ epsilon)
                    model.addConstr(expr, GRB.GREATER_EQUAL, label_budget-epsilon)    
                    

                try:
                    model.optimize()
                except e:
                    print(e)
                with open('../../../../Git/influence_maximization/experiments/im500/results/fairmip/fairseed/base100/'+attribute+'/output_'+str(index)+'.txt', "w") as of:    
                    for key,value in var_seed_dict.items():
                        if(value.x > 0):
                            print(key, file=of)

In [3]:
MIP_IM()

Optimize a model with 100005 rows, 50500 columns and 311699 nonzeros
Variable types: 0 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 59653 rows and 29010 columns
Presolve time: 1.07s
Presolved: 40352 rows, 21490 columns, 165141 nonzeros
Variable types: 0 continuous, 21490 integer (21490 binary)
Found heuristic solution: objective 7698.0000000

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 2 rows and 0 columns
Presolved: 40350 rows, 21492 columns, 164643 nonzeros

Concurrent spin time: 0.53s

Solved with dual simplex

Root relaxation: objective 7.911000e+03, 31776 iterations, 3.45 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0        

Presolve time: 1.03s
Presolved: 40907 rows, 21781 columns, 164669 nonzeros
Variable types: 0 continuous, 21781 integer (21781 binary)
Found heuristic solution: objective 7531.0000000

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 2 rows and 0 columns
Presolved: 40905 rows, 21783 columns, 164171 nonzeros

Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 7.715000e+03, 29919 iterations, 2.91 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7715.0000000 7715.00000  0.00%     -    4s

Explored 0 nodes (29929 simplex iterations) in 4.71 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 7715 7531 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.715000000000e+03, best bound 7.715000000000e+03, gap 0.0000%
Optimize a model w

Presolved: 40275 rows, 21427 columns, 163932 nonzeros

Concurrent spin time: 0.93s

Solved with dual simplex

Root relaxation: objective 7.680000e+03, 33717 iterations, 3.31 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7680.0000000 7680.00000  0.00%     -    5s

Explored 0 nodes (33717 simplex iterations) in 5.20 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 7680 7524 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.680000000000e+03, best bound 7.680000000000e+03, gap 0.0000%
Optimize a model with 100005 rows, 50500 columns and 310820 nonzeros
Variable types: 0 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 59139 rows and 28667 colu


Concurrent spin time: 0.70s

Solved with dual simplex

Root relaxation: objective 7.457000e+03, 31391 iterations, 3.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7457.0000000 7457.00000  0.00%     -    4s

Explored 0 nodes (31391 simplex iterations) in 4.69 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 7457 7301 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.457000000000e+03, best bound 7.457000000000e+03, gap 0.0000%
Optimize a model with 100005 rows, 50500 columns and 312446 nonzeros
Variable types: 0 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 59017 rows and 28628 columns
Presolve time: 1.06s
Presolved: 40988 rows, 21872 

Presolve removed 60077 rows and 29206 columns
Presolve time: 1.05s
Presolved: 39938 rows, 21294 columns, 162149 nonzeros
Variable types: 0 continuous, 21294 integer (21294 binary)
Found heuristic solution: objective 7259.0000000

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 7 rows and 0 columns
Presolved: 39931 rows, 21301 columns, 161656 nonzeros

Concurrent spin time: 0.25s

Solved with dual simplex

Root relaxation: objective 7.637000e+03, 31413 iterations, 2.81 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7637.0000000 7637.00000  0.00%     -    4s

Explored 0 nodes (31413 simplex iterations) in 4.84 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 7637 7259 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.637000000000e+03, best bound 7.63

...

  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 2e+01]
Presolve removed 59471 rows and 28903 columns
Presolve time: 1.08s
Presolved: 40556 rows, 21597 columns, 160650 nonzeros
Variable types: 0 continuous, 21597 integer (21597 binary)
Found heuristic solution: objective 7554.0000000

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 2 rows and 0 columns
Presolved: 40554 rows, 21599 columns, 160243 nonzeros

Concurrent spin time: 0.52s

Solved with dual simplex

Root relaxation: objective 7.813833e+03, 31343 iterations, 2.67 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7813.83333    0 1480 7554.00000 7813.83333  3.44%     -    4s
H    0     0                    7742.0000000 7813.83333  0.93%     -    4s
H    0     0                    7811.0000000 7813.83333 

Thread count was 8 (of 8 available processors)

Solution count 2: 7949 7615 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.949000000000e+03, best bound 7.949000000000e+03, gap 0.0000%
Optimize a model with 100011 rows, 50500 columns and 311522 nonzeros
Variable types: 0 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 2e+01]
Presolve removed 59454 rows and 28955 columns
Presolve time: 1.06s
Presolved: 40557 rows, 21545 columns, 165262 nonzeros
Variable types: 0 continuous, 21545 integer (21545 binary)
Found heuristic solution: objective 7599.0000000

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 3 rows and 0 columns
Presolved: 40554 rows, 21548 columns, 164801 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   37028    7


Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 4 rows and 0 columns
Presolved: 41235 rows, 21989 columns, 166093 nonzeros

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 8.445000e+03, 30638 iterations, 2.82 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    8445.0000000 8445.00000  0.00%     -    4s

Explored 0 nodes (30638 simplex iterations) in 4.51 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 8445 8128 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.445000000000e+03, best bound 8.445000000000e+03, gap 0.0000%
Optimize a model with 100011 rows, 50500 columns and 311237 nonzeros
Variable types: 0 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective ran


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    8129.0000000 8129.00000  0.00%     -    4s

Explored 0 nodes (31126 simplex iterations) in 4.73 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 8129 7904 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.129000000000e+03, best bound 8.129000000000e+03, gap 0.0000%
Optimize a model with 100011 rows, 50500 columns and 312920 nonzeros
Variable types: 0 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 2e+01]
Presolve removed 59310 rows and 28805 columns
Presolve time: 1.11s
Presolved: 40701 rows, 21695 columns, 166168 nonzeros
Variable types: 0 continuous, 21695 integer (21695 binary)
Found heuristic solution: objective 7499.000

Presolve removed 59604 rows and 28983 columns
Presolve time: 1.02s
Presolved: 40407 rows, 21517 columns, 164262 nonzeros
Variable types: 0 continuous, 21517 integer (21517 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 5 rows and 0 columns
Presolved: 40402 rows, 21522 columns, 163767 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   34630    7.5690326e+03   0.000000e+00   9.217768e+01      5s
   34777    7.5810000e+03   0.000000e+00   0.000000e+00      5s
   34777    7.5810000e+03   0.000000e+00   0.000000e+00      5s
Concurrent spin time: 0.22s

Solved with dual simplex

Root relaxation: objective 7.581000e+03, 34021 iterations, 3.24 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7581.0000000 7581.00000  0.00%     -    5s

Exp